In [43]:
import os, json
import jsonlines

DATA_DIR = "../data"
data_list = ["KoAlpaca/KoAlpaca_v1.1.jsonl", "KoChatGPT/kochatgpt_1_SFT.jsonl", "korquad-chat-v1/korquad-chat.json", "OIG-small-chip2-ko/oig-smallchip2-dedu.jsonl", "ShareGPT_DeepL/ko_alpaca_style_dataset.json", "KoAlpaca/ko_alpaca_data.json"]

### KoAlpaca v1.0 dataset

In [44]:
data_path = data_list[-1]

with open(os.path.join(DATA_DIR, data_path), "r") as f:
    koalpaca_v1_0 = json.load(f)

len(koalpaca_v1_0)

49620

### KoAlpaca v1.1 dataset

In [45]:
data_path = data_list[0]

koalpaca_v1_1 = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        koalpaca_v1_1.append(line)

len(koalpaca_v1_1)

21155

### KochatGPT SFT dataset

In [46]:
data_path = data_list[1]
with open(os.path.join(DATA_DIR, data_path), "r") as f:
    kochatgpt = json.load(f)

len(kochatgpt)

12000

### Korquad dataset

In [47]:
data_path = data_list[2]

korquad = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        korquad.append(line)

len(korquad)

9619

### OIG dataset

In [48]:
data_path = data_list[3]

oig = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        oig.append(line)

len(oig)

210282

### ShareGPT Korean dataset

In [49]:
data_path = data_list[4]

with open(os.path.join(DATA_DIR, data_path), "r") as f:
    sharegpt = json.load(f)

len(sharegpt)

85619

In [50]:
print(koalpaca_v1_0[0])
print(koalpaca_v1_1[0])
print(kochatgpt[0])
print(korquad[0])
print(oig[0])
print(sharegpt[0])

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.', 'input': '', 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}
{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?', 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}
{'prompt': '불고기용 고기 한우에요?', 'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.", 'tokens': 193}
{'source': 'korquad-chat', 'text': '<sys>1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 

In [51]:
from tqdm import tqdm

print("Unifying except for Korquad dataset")
unified_data = koalpaca_v1_1.copy()

# kochatgpt
for sample in tqdm(kochatgpt, total=len(kochatgpt)):
    unified_data.append({"instruction": sample["prompt"], "input":"", "output": sample["completion"]})

# OIG
for sample in tqdm(oig, total=len(oig)):
    unified_data.append({"instruction": sample["user_translated"], "input":"", "output": sample["chip2_translated"]})

# ShareGPT
for sample in tqdm(sharegpt, total=len(sharegpt)):
    unified_data.append({"instruction": sample["instruction"], "input":"", "output": sample["output"]})
    
unified_data.extend(koalpaca_v1_0)
    
len(unified_data)

Unifying except for Korquad dataset


100%|██████████| 85619/85619 [00:00<00:00, 1016143.18it/s]


378676

### Processing Koquard dataset

In [52]:
for sample in tqdm(korquad, total=len(korquad)):
    try:
        sys, usr, bot = sample["text"].split("\n")[:3]
        context = sys.split(">")[1].strip()
        question = usr.split(">")[1].strip()
        answer = bot.split(">")[1].strip()
        unified_data.append({"instruction": question, "input": context, "output": answer})
    except:
        pass

100%|██████████| 9619/9619 [00:00<00:00, 158026.71it/s]


In [53]:
len(unified_data)

387987

In [54]:
import json

with open("../data/unified_instruction_nc.json", "w", encoding="utf-8") as f:
    json.dump(unified_data, f, ensure_ascii=False) # ensure_ascii로 한글이 깨지지 않게 저장

In [55]:
unified_data[-1]

{'instruction': '메탈로-베타락타마제가 어디서 발견되었나요?',
 'input': '유전자의 이름은 인도의 수도 뉴델리의 이름을 따 붙여졌는데, 이는 2009년 용 (Yong) 등이 처음 기록한 이 케이스의 환자가 인도에서 처음 박테리아 감염을 확인하였기 때문이다. 박테리아 감염 치료가 뉴델리 병원에서 실패로 끝난 후 그는 모국 스웨덴으로 송환되었으며, 그곳에서 카바페넴에 내성을 보이는 폐렴막대균의 유전자가 분리되었다. 용 등은 이 내성 메커니즘이 "분명히 인도에서 생겨났으나, 이것이 얼마나 퍼져있는지를 보여주는 데이터가 인도에 존재한다"라고 주장하였다. 보고된 바에 따르면 이 유전자는 파키스탄, 인도, 그리고 여러 아시아 국가에서 발견되었으며 이곳 병원에 입원했던 사람들에 의해 유럽으로 옮겨졌다. 그 중 대다수는 저가의 성형 수술을 위해 의료 관광을 떠났다가 수술 중 감염이 된 사람들이었다.',
 'output': '이 유전자는 파키스탄, 인도, 그리고 여러 아시아 국가에서 발견된 것으로 보고되었습니다.'}